We test the method for finding the Koopman tensor on a simple system where action $u=0$ halves the state ($(2,4)\to (1,2)$) and action $u=1$ doubles the state $((2,4)\to (4,8))$. Furthermore, note that in the OLS calculations below, we made use of the following equality
$$\min_{M} \sum_{i=1}^N \left\|  M \left( \psi(u_i)\otimes  \phi(x_i)\right) - \phi(x_i')  \right\|^2 = \min_{M}\left\lVert M\left(\psi(u_1) \otimes \phi(x_1),\cdots , \psi(u_n) \otimes \phi(x_n) \right)-\left(\phi(x'_1),\cdots , \phi(x'_n) \right)\right\rVert _F$$

where, in our application, $\phi(x) = x$ and $\psi(u)$ is the one-hot indicator function.

In [29]:
import numpy as np

In [30]:
def ols(X, Y, pinv=True):
    if pinv:
        return np.linalg.pinv(X.T @ X) @ X.T @ Y
    return np.linalg.inv(X.T @ X) @ X.T @ Y

In [31]:
# System dynamics
True_K0 = np.array([[0.5, 0], [0, 0.5]])
True_K1 = np.array([[2, 0], [0, 2]])
def f(x, u):
    if u == 0:
        return True_K0 @ x
    return True_K1 @ x

In [32]:
# Hotkey feature map for two actions
def psi(u):
    if u == 0:
        return np.array([[1], [0]])
    return np.array([[0], [1]])

N = 1000 # sample size
# Note the system is linear in the state so there is no need for a feature map for x
d_x = 2
d_psi = 2
x0 = np.array([[2],[4]])
X = np.empty((x0.shape[0],N+1))
X[:, 0] = x0[:,0]
U = np.empty((1,N))
Psi = np.empty((d_psi, N))
kronMatrix = np.empty((d_psi*d_x, N))


for i in range(N):
    U[0, i] = np.round(np.random.uniform(0,1))
    Psi[:, i] = psi(U[0,i])[:, 0]
    X[:, i+1] = f(X[:,i], U[0,i])
    kronMatrix[:, i] = np.kron(X[:, i], Psi[:, i])

X_prime = np.roll(X, -1, axis=1)[:,:-1]
M =  ols(kronMatrix.T, X_prime.T).T




In [33]:
# Candidate reshaped Koopman tensors
K_Fortran_order = np.empty((d_x, d_x, d_psi))
K_C_order = np.empty((d_x, d_x, d_psi))

for i in range(d_x):
    K_Fortran_order[i] = M[i].reshape((d_x,d_psi), order = 'F')
    K_C_order[i] = M[i].reshape((d_x,d_psi), order = 'C')

In [34]:
# Compare derived action depenedent koopman operator with true operator
def K_u(K, u):
    return np.einsum('ijz,z->ij', K, psi(u)[:,0])



In [35]:
True_K1

array([[2, 0],
       [0, 2]])

In [36]:
True_K0

array([[0.5, 0. ],
       [0. , 0.5]])

In [37]:
K_u(K_C_order,1)

array([[0.4, 0.8],
       [0.8, 1.6]])

In [38]:
K_u(K_Fortran_order,1)


array([[0.2, 0.8],
       [0.4, 1.6]])

In [39]:
K_u(K_Fortran_order,0)

array([[0.1, 0.4],
       [0.2, 0.8]])

In [40]:
K_u(K_C_order,0)


array([[0.1, 0.2],
       [0.2, 0.4]])

Note the differences between the true $K$'s and the estimated $K$'s regardless of the reshape and the action chosen. We are unsure of how to remedy this given the current theoretical exposition.